In [4]:
import pybamm as pb
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# fetch parameter values
OKane2022 = pb.ParameterValues("OKane2022")     # original OKane2022 parameters
parameter_values = pb.ParameterValues("OKane2022")        # parameter set to be used and updated
ORegan2022 = pb.ParameterValues("ORegan2022")   # original ORegan2022 parameters

# replace the thermal related parameters in OKane2022 with the ones in ORegan
for param in parameter_values:
    try:
        if OKane2022[param] != ORegan2022[param]:
            parameter_values.update({param: ORegan2022[param]})
        else:
            continue
    except:
        print("There is no value for this parameter")
    #print(parameter_values)

# write the results to a text file to be read

f = open("parameter_values.txt", "a")
sets = [OKane2022, parameter_values, ORegan2022]
all_params = sorted({key for d in sets for key in d})
output_dict = {param: [] for param in all_params}

for param in all_params:
    for d in sets:
        output_dict[param].append(d.get(param, ''))

 # Write each parameter row
for param in all_params:
    values = [str(v) for v in output_dict[param]]
    f.write(f"{param}\t" + "\t".join(values) + "\n")

f.close()

# updates some of the parameter to Li2024 paper

Para_All = []
param = pb.ParameterValues("OKane2022")
para_dict = {
    "Positive electrode LAM constant proportional term [s-1]": 1e-16,
    #'Initial electrolyte excessive amount ratio':1.0, 
    "Contact resistance [Ohm]": 0.0115, # 0.016, # 
    #'Inner SEI lithium interstitial diffusivity [m2.s-1]':2.35555E-18, 
    'SEI lithium interstitial diffusivity [m2.s-1]':2.35555E-18, 

    #'Outer SEI partial molar volume [m3.mol-1]':4E-05,
    "SEI partial molar volume [m3.mol-1]": 4E-05,
    "SEI growth activation energy [J.mol-1]":1e4,
    #"Negative electrode diffusivity activation energy [J.mol-1]": 6e4,
    "Negative particle diffusivity activation energy [J.mol-1]": 6e4,
    #"Positive electrode diffusivity activation energy [J.mol-1]": 1.2e4,
    "Positive particle diffusivity activation energy [J.mol-1]": 1.2e4,
}
for key, value in para_dict.items():
    param.update({key: value})

There is no value for this parameter
{'Ambient temperature [K]': 298.15,
 'Boltzmann constant [J.K-1]': 1.380649e-23,
 'Bulk solvent concentration [mol.m-3]': 2636.0,
 'Cation transference number': 0.2594,
 'Cell cooling surface area [m2]': 0.00531,
 'Cell thermal expansion coefficient [m.K-1]': 1.1e-06,
 'Cell volume [m3]': 2.42e-05,
 'Contact resistance [Ohm]': 0,
 'Current function [A]': 5.0,
 'Dead lithium decay constant [s-1]': 1e-06,
 'Dead lithium decay rate [s-1]': <function SEI_limited_dead_lithium_OKane2022 at 0x000001A3F4AB9760>,
 'EC diffusivity [m2.s-1]': 2e-18,
 'EC initial concentration in electrolyte [mol.m-3]': 4541.0,
 'Electrode height [m]': 0.065,
 'Electrode width [m]': 1.58,
 'Electrolyte conductivity [S.m-1]': <function electrolyte_conductivity_Nyman2008_arrhenius at 0x000001A3F4ABB100>,
 'Electrolyte diffusivity [m2.s-1]': <function electrolyte_diffusivity_Nyman2008_arrhenius at 0x000001A3F4ABAFC0>,
 'Electron charge [C]': 1.602176634e-19,
 'Exchange-current den

KeyError: "Cannot update parameter 'Negative electrode diffusivity activation energy [J.mol-1]' as it does not have a default value. (Negative particle diffusivity activation energy [J.mol-1]). If you are sure you want to update this parameter, use param.update({name: value}, check_already_exists=False)"

parameter_values = merged OKane2022 + ORegan2022

param = Li2024 parameters

In [ ]:
# set mesh size for model
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 80,  # negative particle
    "r_p": 20,  # positive particle
}

# cycling protocol
cycles = 10
exp = pb.Experiment(
    # initial
    [
        "Hold at 4.2V until C/100",
        "Rest for 4 hours",
        "Discharge at 0.1C until 2.5V",    # initial capacity check
        "Charge at 0.3C until 4.2V",
        "Hold at 4.2V until C/100",
    ]
    + [
        (
            "Discharge at 1C until 2.5V",   # aging cycles
            "Charge at 0.3C until 4.2V",
        )
    ] *cycles
    + ["Discharge at 0.1C until 2.5V"],     # capacity check
)

In [ ]:
SEI_only = [
    "SEI": "interstitial-diffusion limited",
    #"SEI": "ec reaction limited",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
]

SEI_plating = [
    "SEI": "interstitial-diffusion limited",
    "SEI": "ec reaction limited",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",

    "lithium plating": "partially reversible",
    "lithium plating porosity": "true"
    "lithium plating porosity change":"false",
]

SEI_plating_thermal = [
    "SEI": "interstitial-diffusion limited",
    "SEI": "ec reaction limited",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",

    "lithium plating": "partially reversible",
    "lithium plating porosity": "true"
    "lithium plating porosity change":"false",

    "thermal": "lumped"
]

model_sei_only = pb.lithium_ion.DFN(
    options=SEI_only,
)

In [ ]:
model_sei_hy = pb.lithium_ion.DFN(
    options={
    "SEI": "interstitial-diffusion limited",
    "SEI on cracks": "true",
    
    "lithium plating": "none",
    "lithium plating porosity change":"false",
    "particle mechanics": "constant cracks",
    "loss of active material": "none",

    "contact resistance": "true",
    "open-circuit potential": "current sigmoid",
    "SEI film resistance": "distributed", 
    "SEI porosity change": "true",
    "thermal": "lumped",  
    }, )


Para_All = []
param = pb.ParameterValues("OKane2023")
para_dict = {
    "Positive electrode LAM constant proportional term [s-1]": 1e-16,
    #'Initial electrolyte excessive amount ratio':1.0, 
    "Contact resistance [Ohm]": 0.0115, # 0.016, # 
    'Inner SEI lithium interstitial diffusivity [m2.s-1]':2.35555E-18, 

    'Outer SEI partial molar volume [m3.mol-1]':4E-05,
    "SEI growth activation energy [J.mol-1]":1e4,
    "Negative electrode diffusivity activation energy [J.mol-1]":6e4,
    "Positive electrode diffusivity activation energy [J.mol-1]":1.2e4,
}
for key, value in para_dict.items():
    param.update({key: value})

V_max = 4.2;        V_min = 2.5
exp_topup_text = [ (
    f"Discharge at 0.5C until {V_max-0.2}V", # start from discharge as it is easier for unbalanced cells
    f"Charge at 0.3C until {V_max}V",
    f"Hold at {V_max}V until C/100",
    "Rest for 1 hours", 
    ) ] 
exp_GITT_text = [ (
    "Rest for 60 seconds (1 second period)", 
    "Discharge at 1C for 144 seconds or until 2.5V (0.1 second period)", 
    "Rest for 1 hour (1 second period)"
    ) ]

experiment = pb.Experiment( 
    exp_topup_text * 1 + exp_GITT_text * 25 ) 
var_pts = {
    "x_n": 5,  # negative electrode
    "x_s": 5,  # separator 
    "x_p": 5,  # positive electrode
    "r_n": 80,  # negative particle
    "r_p": 20,  # positive particle
}


In [ ]:
parameter_values.search("temperature")

In [ ]:
# comparison of different SEI growth mechanisms 
# coupling together SEI and thermal submodels

SEI_options = [
    "reaction limited",
    "solvent-diffusion limited",
    "electron-migration limited",
    "interstitial-diffusion limited",
    "ec reaction limited",
]
SEI_sols = []

for option in SEI_options:
    coupled_model = pb.lithium_ion.DFN(
        options={
            "SEI": option,
            "SEI porosity change": "true",
            "thermal": "lumped",        # homogeneous within the battery
        }
    )
    sim = pb.Simulation(
        coupled_model,
        experiment=exp,
        parameter_values=parameter_values,
        var_pts=var_pts,
    )
    SEI_sols.append(sim.solve())
    print(option, ":", coupled_model.variable_names())

    
    
output_variables = [
    "Loss of capacity to negative SEI [A.h]",
    'Negative SEI thickness [m]',
    "Voltage [V]",
    'Electrolyte potential [V]',
    "X-averaged negative electrode porosity",
    'Loss of capacity to negative SEI [A.h]',
]
pb.dynamic_plot(
    SEI_sols,
    output_variables=output_variables
)

coupled_model.print_parameter_info()

In [ ]:
# parametric sweep through different temperatures
# setting up the solver
solver = pb.IDAKLUSolver()

# empty solution arrays
solutions = []
Qt = []                 # Throughput capacity 
Q_SEI = []              # Loss of capacity to negative SEI
C_dis = []              # Discharge capacity [A.h]
I_exchange_anode = []   # Negative electrode exchange current density [A.m-2]
I_exchange_cathode = [] # Positive electrode exchange current density [A.m-2]

# parametric study of effect of temperature

temps = [-15, 5, 25, 55, 125, 145]
for temp in temps:
    temp = temp + 273.15    # convert to Kelvin
    params.update({
        "Ambient temperature [K]" : temp,
        "Initial temperature [K]" : temp,
    })
    params.search("temperature")

    sim = pb.Simulation(
        coupled_model,
        parameter_values=params,
        experiment=exp,
        solver=solver,
        var_pts=var_pts,
    )
    sol = sim.solve()
    #print(parametric_sols["Loss of capacity to negative SEI [A.h]"].entries)
    solutions.append(sim.solve())

    # sorting outputs into variables
    Qt.append(sol["Throughput capacity [A.h]"].entries)
    Q_SEI.append(sol["Loss of capacity to negative SEI [A.h]"].entries)
    C_dis.append(sol["Discharge capacity [A.h]"].entries)
    I_exchange_anode.append(sol["Negative electrode exchange current density [A.m-2]"].entries)
    I_exchange_cathode.append(sol["Positive electrode exchange current density [A.m-2]"].entries)


    Q_plating = sol["Loss of capacity to negative lithium plating [A.h]"].entries
    Q_side = sol["Total capacity lost to side reactions [A.h]"].entries
    Q_LLI = (sol["Total lithium lost [mol]"].entries * 96485.3 / 3600)  # convert from mol to A.h

    LLI = sol["Loss of lithium inventory [%]"].entries
    LAM_neg = sol["Loss of active material in negative electrode [%]"].entries
    LAM_pos = sol["Loss of active material in positive electrode [%]"].entries

    eps_neg_avg = sol["X-averaged negative electrode porosity"].entries
    eps_neg_sep = sol["Negative electrode porosity"].entries[-1, :]
    eps_neg_CC = sol["Negative electrode porosity"].entries[0, :]



In [ ]:
# plotting loss of capacity to negative SEI against Q_tot
plt.figure()
for i in range(0, len(temps)-1):
    plt.plot(Qt[i], Q_SEI[i], label=f"Ambient temperature {temps[i]} C", linestyle="dashed")
    plt.xlabel("Throughput capacity [A.h]")
    plt.ylabel("Loss of capacity to negative SEI [A.h]")
plt.legend()
plt.title("Solvent-limited SEI growth only")
plt.show()

# plotting the discharge capacities against Q_tot
plt.figure()
for i in range(0, len(temps)-1):
    plt.plot(Qt[i], C_dis[i], label=f"Ambient temperature {temps[i]} C", linestyle="dashed")
    plt.xlabel("Throughput capacity [A.h]")
    plt.ylabel("Discharge capacity [A.h]")
plt.legend()
plt.title("Solvent-limited SEI growth only")
plt.show()

# plotting the electrode exchange current densities


In [ ]:
Qt = sol["Throughput capacity [A.h]"].entries
Q_SEI = sol["Loss of capacity to negative SEI [A.h]"].entries
#Q_SEI_cr = sol["Loss of capacity to negative SEI on cracks [A.h]"].entries
Q_plating = sol["Loss of capacity to negative lithium plating [A.h]"].entries
Q_side = sol["Total capacity lost to side reactions [A.h]"].entries
Q_LLI = (
    sol["Total lithium lost [mol]"].entries * 96485.3 / 3600
)  # convert from mol to A.h
plt.figure()
plt.plot(Qt, Q_SEI, label="SEI", linestyle="dashed")
#plt.plot(Qt, Q_SEI_cr, label="SEI on cracks", linestyle="dashdot")
plt.plot(Qt, Q_plating, label="Li plating", linestyle="dotted")
plt.plot(Qt, Q_side, label="All side reactions", linestyle=(0, (6, 1)))
plt.plot(Qt, Q_LLI, label="All LLI")
plt.xlabel("Throughput capacity [A.h]")
plt.ylabel("Capacity loss [A.h]")
plt.legend()
plt.show()

Qt = sol["Throughput capacity [A.h]"].entries
LLI = sol["Loss of lithium inventory [%]"].entries
LAM_neg = sol["Loss of active material in negative electrode [%]"].entries
LAM_pos = sol["Loss of active material in positive electrode [%]"].entries
plt.figure()
plt.plot(Qt, LLI, label="LLI")
plt.plot(Qt, LAM_neg, label="LAM (negative)")
plt.plot(Qt, LAM_pos, label="LAM (positive)")
plt.xlabel("Throughput capacity [A.h]")
plt.ylabel("Degradation modes [%]")
plt.legend()
plt.show()

eps_neg_avg = sol["X-averaged negative electrode porosity"].entries
eps_neg_sep = sol["Negative electrode porosity"].entries[-1, :]
eps_neg_CC = sol["Negative electrode porosity"].entries[0, :]
plt.figure()
plt.plot(Qt, eps_neg_avg, label="Average")
plt.plot(Qt, eps_neg_sep, label="Separator", linestyle="dotted")
plt.plot(Qt, eps_neg_CC, label="Current collector", linestyle="dashed")
plt.xlabel("Throughput capacity [A.h]")
plt.ylabel("Negative electrode porosity")
plt.legend()
plt.show()

sol.plot(["X-averaged electrolyte concentration [mol.m-3]"])